### **1. <u>Project Setup<u/>:**

In [1]:
!pip install gdown

In [2]:
!pip install torch torchvision numpy matplotlib pandas scikit-learn pillow

In [3]:
import torch 
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim 

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device available: {device}")

Device available: cpu


In [4]:
# Data Transformation

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])   

#### ***Code Explainer***
- **`transforms.Compose`:** Creates a composition of multiple image transformations to be applied to the images.
- **`transforms.Resize((224, 224))`:** Resizes each image to **224x224 pixels**.
- **`transforms.RandomHorizontalFlip()`:** Applies horizontal flip to the images randomly. This helps in augmenting the dataset to improve generalization (not applied in the second set of transformations).
- **`transforms.RandomRotation(10)`:** Rotates the images by a degree randomly selected from **(-10, 10)**, which again helps in dataset augmentation (not applied in the second set of transformations).
- **`transforms.ToTensor()`:** Converts the images to PyTorch tensors (from *PIL images* or *NumPy arrays*).
- **`transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])`:** Normalizes the tensor image with mean and standard deviation provided. These values are standard normalization means and stds for pre-trained models on *ImageNet*.

In [5]:
import gdown

file_id = '1pcTNgOsIHAgLzHBtMBoV0y-4fVMnvoNT'
output = 'archive.zip'  # Replace with the desired path where you want to save the downloaded file
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1pcTNgOsIHAgLzHBtMBoV0y-4fVMnvoNT
From (redirected): https://drive.google.com/uc?id=1pcTNgOsIHAgLzHBtMBoV0y-4fVMnvoNT&confirm=t&uuid=8662600d-dd2e-434c-b9f4-a749d245a9f5
To: C:\Users\jatin\Desktop\S3\Internship\RA-AIHI\Assignments\T2\archive.zip
 80%|███████▉  | 7.80G/9.77G [1:00:19<15:13, 2.15MB/s]


'archive.zip'

In [6]:
import zipfile 

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall('C:/Users/jatin/Desktop/S3/Internship/RA-AIHI/Assignments/T2')

NameError: name 'file' is not defined

### **2. <u>Model Development<u/>:**

In [ ]:
# Defining the model 
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 9)
model = model.to(device)

#### ***Code Explainer***
- **`models.resnet50(pretrained=True)`:** Loads a pre-trained *ResNet50 model* from PyTorch's model zoo, pre-trained on *ImageNet*.
- **`num_ftrs = model.fc.in_features`:** Retrieves the number of input features for the final fully connected (*fc*) layer of the model.
- **`model.fc = nn.Linear(num_ftrs, 9)`:** Replaces the existing fully connected layer with a new one that has **9 output features** (corresponding to 9 classes in your dataset).
- **`model = model.to(device)`:** Moves the model to the specified device (GPU or CPU), allowing for computations to be performed on that device. This improves performance if a GPU is available.

In [ ]:
# Loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### ***Code Explainer***
- **`criterion = nn.CrossEntropyLoss()`:** Defines the loss function as cross-entropy loss, which is commonly used for multi-class classification tasks. This loss function combines *Log Softmax* and *NLL Loss* in a single function.
- **`optimizer = optim.Adam(model.parameters(), lr=0.001)`:** Sets up the Adam optimizer for adjusting model weights based on the calculated gradients, with a **learning rate** of **0.001**. Adam is an adaptive learning rate optimization algorithm that combines the benefits of AdaGrad and RMSProp to handle sparse gradients on noisy problems.

In [ ]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader: 
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  
        outputs = model(inputs)
        loss = criterion(outputs, labels) 
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

#### ***Code Explainer***
- **`for epoch in range(num_epochs)`:** Initiates a loop over a specified number of epochs, allowing the model to learn from the data multiple times.
- **`for inputs, labels in train_loader`:** Iterates over the training data loader, which supplies batches of inputs and corresponding labels.
- **`inputs, labels = inputs.to(device), labels.to(device)`:** Moves the input data and labels to the configured device (either CPU or GPU), ensuring that the model operations are performed on the right device.
- **`optimizer.zero_grad()`:** Clears old gradients from the last step before performing a new optimization step. This is necessary because gradients accumulate by default.
- **`outputs = model(inputs)`:** Passes the input data through the model to get predictions.
- **`loss = criterion(outputs, labels)`:** Computes the loss between the model's predictions and the actual labels using the cross-entropy loss function.
- **`loss.backward()`:** Computes the derivative of the loss with respect to the parameters (gradient) using backpropagation.
- **`optimizer.step()`:** Performs a single optimization step (parameter update).
- **`running_loss += loss.item()`:** Aggregates the loss over each batch for monitoring.
- **`print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')`:** Prints the average loss for each epoch, providing insight into how well the model is learning from the training data.

In [ ]:
# Validation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:  # Corrected 'from' to 'for' and 'label' to 'labels'
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Corrected 'output' to 'outputs'
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Validation accuracy: {100 * correct / total}%')  # Added calculation for percentage

#### ***Code Explainer***
- **`model.eval()`:** Sets the model to evaluation mode. This is important as it tells the model to behave in an inference mode (e.g., turning off dropout).
- **`with torch.no_grad()`:** Disables gradient calculation, which is beneficial for inference since it reduces memory usage and speeds up computation.
- **`for images, labels in val_loader`:** Iterates over the validation data loader, which provides batches of images and labels for validation.
- **`images, labels = images.to(device), labels.to(device)`:** Moves the images and labels to the appropriate device (CPU or GPU), ensuring computations are performed where expected.
- **`outputs = model(images)`:** Computes the model's output predictions for the given batch of images.
- **`_, predicted = torch.max(outputs.data, 1)`:** Finds the predicted class labels by selecting the class with the maximum output in the logits dimension.
- **`total += labels.size(0)`:** Accumulates the total number of labels processed, which is used to calculate accuracy.
- **`correct += (predicted == labels).sum().item()`:** Increments the correct predictions counter by the number of correct predictions in the batch.
- **`print(f'Validation accuracy: {100 * correct / total}%')`:** Prints the validation accuracy as a percentage of correct predictions over the total number of predictions.

In [ ]:
torch.save(model.state_dict(), 'best_model.pth')

### **3. <u>Evaluation & Tuning<u/>:**

### **4. <u>Deployment and Testing<u/>:**